<div dir="rtl" style="text-align:right">

## م2.5 — إنشاء تحليل بيانات مناخية قابل لإعادة الإنتاج

جزء من: <bdi dir="ltr"><a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science">Computational Climate Science</a></bdi> | **الدرس السابق** | **الدرس التالي**

**المحتويات:**

- <a href="#Creating-a-reproducible-workflow">إنشاء سير عمل قابل لإعادة الإنتاج</a>
  - <a href="#Our-workflow:-Downloading-the-data-(Step-1)">سير عملنا: تنزيل البيانات (الخطوة 1)</a>
  - <a href="#Our-workflow:-Data-processing-(Step-2)">سير عملنا: معالجة البيانات (الخطوة 2)</a>
- <a href="#A-reproducible-project's-files">ملفات مشروع قابل لإعادة الإنتاج</a>
- <a href="#Comparing-multiple-years-of-climate-data">مقارنة عدة سنوات من بيانات المناخ</a>
  - <a href="#Computing-TOA-radiation">حساب إشعاع أعلى الغلاف الجوي (TOA)</a>
  - <a href="#Computing-PET">حساب PET</a>
  - <a href="#Resampling-the-CHIRPS-data">إعادة أخذ عينات بيانات CHIRPS</a>
  - <a href="#Selecting-part-of-an-xarray-time-series">اختيار جزء من سلسلة زمنية في xarray</a>

</div>


---

<div dir="rtl" style="text-align:right">

## إنشاء سير عمل قابل لإعادة الإنتاج

في الدرس السابق استخدمنا <bdi dir="ltr">dask</bdi> و<bdi dir="ltr">xarray</bdi> لقراءة مجموعة من ملفات <bdi dir="ltr">netCDF</bdi>، ثم قمنا **بتمرير** (mapping) دالة **متجهة** (vectorized) على كل مصفوفة ضمن سلسلة زمنية. وأنتجنا رسمًا يوضح نسبة **الهطول إلى PET** لتيارت (الجزائر) في الأشهر الأولى من <bdi dir="ltr">2024</bdi> خلال جفاف شديد.

لوحده، الرسم لا يجيبنا: ما مدى شدة الجفاف في تيارت؟ رغم أن الهطول في المنطقة عوّض أقل من <bdi dir="ltr">5%</bdi> من العجز المائي خلال الأشهر الماضية، فقد يكون ذلك جزءًا من الدورة الموسمية الطبيعية. نحن نعرف أن الفترة من يناير إلى أبريل تُعد نسبيًا أكثر رطوبة في تيارت، لكن يبقى السؤال: **هل يمكننا مقارنة هذا العام بسنوات سابقة؟**

كلما أردنا تطبيق تحليل مكتمل على مجموعة بيانات جديدة (عبر الزمن أو المكان) فهذا وقت مناسب لتحسين تمثيل سير العمل لدينا. انظر إلى السكربتين أدناه — فهما يمثلان أول خطوتين في سير عملنا.

</div>


<div dir="rtl" style="text-align:right">

### سير عملنا: تنزيل البيانات (الخطوة 1)

قد يُسمّى الملف الأول مثلًا: **<bdi dir="ltr">YYYYMMDD_step1_download_MERRA2_data.py</bdi>**. تذكّر أن <bdi dir="ltr">YYYYMMDD</bdi> هو تاريخ اليوم، وهو يساعدنا على **ربط ملفات المخرجات بالشيفرة التي أنشأتها**.

لاحظ أن ملفات بايثون ذات الامتداد <bdi dir="ltr">.py</bdi> يمكن أن تحتوي على **توثيق على مستوى الملف** (file-level docstring). في المثال أدناه يبدأ التوثيق بسلسلة متعددة الأسطر <bdi dir="ltr">'''</bdi>. ويُفترض أن يبدأ هذا التوثيق من السطر الأول في الملف. هذا مهم جدًا لسير عمل قابل لإعادة الإنتاج، لأن أول سطر هو أول ما نراه لفهم هدف الملف.

</div>

<div dir="ltr" style="text-align:left">

```python
'''
Downloads MERRA-2 M2SDNXSLV data, for the first 5 months of a year.
Read more about MERRA-2 here:

    https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/

Data are downloaded to this folder:

    data_raw/MERRA2
'''

import earthaccess

DATA_YEAR = 2023

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = (f"{DATA_YEAR}-01-01", f"{DATA_YEAR}-05-31"))

# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')
```

</div>


<div dir="rtl" style="text-align:right">

&#x1F449; **من الأعلى إلى الأسفل، لاحظ ما يلي:**

- وضع **توثيق مستوى الملف** في أعلى السكربت يشرح الهدف، ومراجع إضافية، وكيف يغيّر السكربت بنية المجلدات/الملفات.
- وضع أوامر <bdi dir="ltr">import</bdi> قرب الأعلى يساعد القارئ على معرفة المكتبات المطلوبة. وضعها في أماكن متفرقة يزيد احتمالية حدوث خطأ مفاجئ مثل <bdi dir="ltr">ImportError</bdi> عند التشغيل.
- تعريف المعاملات القابلة للتغيير (مثل السنة) بوضوح في أعلى السكربت وبأحرف كبيرة — مثل <bdi dir="ltr">DATA_YEAR</bdi> — يجعل إعادة تشغيل السكربت لسنوات مختلفة أسهل دون البحث داخل الملف.

</div>


<div dir="rtl" style="text-align:right">

### سير عملنا: معالجة البيانات (الخطوة 2)

الخطوة التالية هي قراءة ملفات البيانات ثم حساب إشعاع أعلى الغلاف الجوي <bdi dir="ltr">(TOA)</bdi>. وقد يُسمّى الملف الثاني مثلًا: **<bdi dir="ltr">YYYYMMDD_step2_compute_TOA_radiation.py</bdi>**.

</div>

<div dir="ltr" style="text-align:left">

```python
'''
Computes top-of-atmosphere (TOA) radiation from a series of MERRA-2 
M2SDNXSLV files, then writes an output netCDF file. TOA radiation is
calculated according to the FAO formula for extraterrestrial radiation:

    https://www.fao.org/4/X0490E/x0490e07.htm#radiation
'''

import numpy as np
import xarray as xr

# NOTE: This will be different on your computer system and you should
#   use an absolute path, not a relative path
MERRA2_DATA_DIR = './data_raw/MERRA2'
DATA_YEAR = 2023
OUTPUT_FILE = f'./outputs/YYYYMMDD_MERRA2_{DATA_YEAR}_with_TOA-radiation.nc'

def main():
    ds = xr.open_mfdataset(f'{MERRA2_DATA_DIR}/*{DATA_YEAR}*.nc4', chunks = 'auto')
    lats = ds['lat'].values.reshape((361, 1, 1))\
        .repeat(ds.lon.size, axis = 1)\
        .repeat(ds.time.size, axis = 2)
    ds['lat_grid'] = (('lat', 'lon', 'time'), lats)

    # Compute TOA radiation
    template = ds['T2MMEAN']
    template.name = 'toa_radiation'
    result = xr.map_blocks(toa_radiation_wrapper, ds, template = template)
    toa_result = result.compute()
    # Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
    ds['toa_radiation'] = toa_result * 0.408
    
    # Write the file to disk, just the important variables
    ds = ds[['T2MMAX', 'T2MMEAN', 'T2MMIN', 'toa_radiation']]
    comp = dict(zlib = True, complevel = 5)
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(OUTPUT_FILE, format = 'NETCDF4', encoding = encoding)

    
def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the earth-sun
    distance; w is the sunset hour angle; and D is the solar declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    
    # Convert latitude from degrees to radians
    lat_radians = np.deg2rad(latitude)
    # Earth-Sun distance, as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos(doy * ((2 * pi) / 365))
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(doy * ((2 * pi) / 365) - 1.39)
    
    # Sunset hour angle; we use np.where() below to guard against
    #   warnings where arccos() would return invalid values, which
    #   happens when the argument is outside [-1, 1]
    _hour_angle = -np.tan(lat_radians) * np.tan(declination)
    _hour_angle = np.where(np.abs(_hour_angle) > 1, np.nan, _hour_angle)
    sunset_hour_angle = np.arccos(_hour_angle)
    
    return ((24 * 60) / pi) * solar_constant * earth_sun_dist *\
        (sunset_hour_angle * np.sin(lat_radians) * np.sin(declination) +
            np.cos(lat_radians) * np.cos(declination) * np.sin(sunset_hour_angle))


def toa_radiation_wrapper(dataset):
    '''
    Wraps toa_radiation to work with an xarray.Dataset

    Parameters
    ----------
    dataset : xarray.Dataset
        Input dataset with "lat_grid" and "time" variables

    Returns
    -------
    xarray.DataArray
    '''
    return toa_radiation(dataset['lat_grid'], dataset['time.dayofyear'])


# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

</div>


<div dir="rtl" style="text-align:right">

&#x1F449; **مرة أخرى، لاحظ ما يلي:**

- وجود **توثيق مستوى الملف** وأوامر <bdi dir="ltr">import</bdi> قرب الأعلى يساعد المستخدم على فهم الهدف ومعرفة المتطلبات.
- دالة <bdi dir="ltr">toa_radiation()</bdi> تحتوي على **توثيق مستوى الدالة** يشرح المدخلات والمخرجات وطريقة الحساب.

&#x1F449; **تأمل السطر الذي يحتوي** <bdi dir="ltr">if __name__ == '__main__'</bdi> **— ماذا يعني؟**

- كل ملف <bdi dir="ltr">.py</bdi> يمكن تشغيله بطريقتين: تشغيله كسكربت عبر <bdi dir="ltr">python myscript.py</bdi> أو استيراده كوحدة عبر <bdi dir="ltr">import myscript</bdi>.
- عند تشغيل الملف كسكربت، تكون قيمة <bdi dir="ltr">__name__</bdi> هي <bdi dir="ltr">'__main__'</bdi>. وعند استيراده كوحدة تكون قيمة <bdi dir="ltr">__name__</bdi> هي اسم الوحدة.

**لماذا يهمنا ذلك؟** لأن استيراد ملف كوحدة يؤدي إلى تنفيذ كل الكود الموجود فيه. لذلك نضع منطق التشغيل داخل <bdi dir="ltr">main()</bdi> ونشغّله فقط عند التشغيل كسكربت.

</div>

<div dir="ltr" style="text-align:left">

```python
# If the file is run as a script, run the main() function
if __name__ == '__main__':
    main()
```

</div>

<div dir="rtl" style="text-align:right">

إذا كان هذا مربكًا الآن، فاعتبره تقنية "سحرية" تسمح لنا بكتابة ملف يمكن تشغيله كسكربت ويمكن استيراده كوحدة أيضًا.

</div>


---

<div dir="rtl" style="text-align:right">

## ملفات مشروع قابل لإعادة الإنتاج

هناك خطوات أخرى في التحليل، لكن نترك لك كتمرين كتابة سكربتات قابلة لإعادة الاستخدام وموثّقة جيدًا مثل المثال أعلاه. قد يبدو مجلد المشروع النهائي مثل هذا:

</div>

![](./assets/M2_file_tree_workflow.png)

<div dir="rtl" style="text-align:right">

&#x1F449; **لاحظ أن كل ملفات** <bdi dir="ltr">scripts</bdi> **تتضمن** <bdi dir="ltr">step1</bdi> أو <bdi dir="ltr">step2</bdi> أو <bdi dir="ltr">step3</bdi> **لتوضيح ترتيب التنفيذ.** هذا يساعدك (ويساعد الآخرين) على تشغيل سير العمل بالترتيب الصحيح حتى لو عدت للمشروع بعد أشهر.

</div>


---

<div dir="rtl" style="text-align:right">

## مقارنة عدة سنوات من بيانات المناخ

في النهاية، نريد الإجابة عن سؤالنا من الدرس السابق: كيف تقارن نسبة **الهطول إلى PET** في شتاء <bdi dir="ltr">2024</bdi> مع شتاء <bdi dir="ltr">2023</bdi> في تيارت؟

سير العمل القابل لإعادة الإنتاج أعلاه يمكن استخدامه للإجابة عن هذا السؤال. كما سنرى كيف نجيب عنه داخل **جلسة بايثون تفاعلية** في هذا الدفتر <bdi dir="ltr">(Jupyter Notebook)</bdi>. إذا كنا قد نزّلنا بيانات <bdi dir="ltr">MERRA-2</bdi> لكلا العامين، فنحن جاهزون لتحميلها باستخدام <bdi dir="ltr">xarray.open_mfdataset()</bdi>.

</div>


In [ ]:
import xarray as xr
from matplotlib import pyplot

ds_chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')
ds_merra2 = xr.open_mfdataset('data_raw/MERRA2/*.nc4', chunks = 'auto')

<div dir="rtl" style="text-align:right">

السكربت <bdi dir="ltr">YYYYMMDD_step2_compute_TOA_radiation.py</bdi> يحتوي على دالة مفيدة قابلة لإعادة الاستخدام اسمها <bdi dir="ltr">toa_radiation()</bdi>. **كيف نستفيد منها دون نسخ ولصق تعريف الدالة داخل سكربت آخر أو داخل هذا الدفتر؟** بشكل عام يجب تجنب وجود أكثر من نسخة للدالة نفسها.

&#x1F449; كما ذكرنا، وضع الشرط <bdi dir="ltr">if __name__ == '__main__':</bdi> يسمح لنا باستيراد السكربت كوحدة دون تشغيل جزء التنفيذ الرئيسي تلقائيًا. ما دام السكربت موجودًا ضمن شجرة الملفات، يمكن استيراده كوحدة. أدناه نستخدم مكتبة <bdi dir="ltr">os</bdi> لاستعراض ملفات المجلد الحالي، وسنرى وجود مجلد <bdi dir="ltr">scripts</bdi>...

</div>


In [ ]:
import os

# Display the files in the current directory
files = os.listdir('.') # The single dot "." indicates the current directory
files.sort()
files

<div dir="rtl" style="text-align:right">

وداخل <bdi dir="ltr">scripts</bdi> يوجد سكربت بايثون الذي نريد استيراده كوحدة.

</div>


In [ ]:
os.listdir('scripts')

<div dir="rtl" style="text-align:right">

كيف يتم ذلك؟ لأن المجلدات وملفات <bdi dir="ltr">.py</bdi> يمكن التعامل معها كوحدات (modules). كل ما نحتاجه هو أمر استيراد مثل المثال أدناه.

</div>


In [ ]:
from scripts.YYYYMMDD_step2_compute_TOA_radiation import toa_radiation_wrapper

<div dir="rtl" style="text-align:right">

لاحظ أننا استوردنا اسمًا واحدًا فقط: <bdi dir="ltr">toa_radiation_wrapper()</bdi>، وهذا يمنحنا الوصول إلى <bdi dir="ltr">toa_radiation()</bdi>. وبفضل توثيق الدالة يمكننا قراءة تفاصيل الاستخدام عبر <bdi dir="ltr">help()</bdi>.

</div>


In [ ]:
help(toa_radiation_wrapper)

<div dir="rtl" style="text-align:right">

### حساب إشعاع أعلى الغلاف الجوي (TOA)

</div>


<div dir="rtl" style="text-align:right">

كما في السابق، لحساب <bdi dir="ltr">TOA</bdi> نحتاج إلى إنشاء شبكة خطوط عرض حتى نستطيع تشغيل الحساب بشكل متجه (vectorized).

</div>


In [ ]:
lat_grid = ds_merra2.lat.values.reshape(1, 361, 1)\
    .repeat(ds_merra2.lon.size, axis = 2)\
    .repeat(ds_merra2.time.size, axis = 0)
lat_grid.shape

In [ ]:
ds_merra2['lat_grid'] = (('time', 'lat', 'lon'), lat_grid)
ds_merra2

<div dir="rtl" style="text-align:right">

ونحتاج أيضًا إلى يوم السنة، وهو متاح مثلًا عبر <bdi dir="ltr">ds_merra2['time.dayofyear']</bdi>.

إذًا نحن جاهزون لحساب <bdi dir="ltr">TOA</bdi>.

</div>


In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
toa_rad = toa_radiation_wrapper(ds_merra2) * 0.408
toa_rad.sel(time = '2024-05-01').plot()

<div dir="rtl" style="text-align:right">

### حساب PET

</div>


<div dir="rtl" style="text-align:right">

سنقوم أيضًا باقتطاع بيانات <bdi dir="ltr">MERRA-2</bdi> لمنطقة تيارت.

</div>


In [ ]:
toa_rad.shape

In [ ]:
import numpy as np

ds_merra2['toa_radiation'] = toa_rad

# Select just the Tiaret region
merra2_tiaret = ds_merra2.sel(lon = -1.32, lat = 35.37, method = 'nearest')

# Compute PET using the Hargreaves equation
merra2_tiaret_pet = 0.0023 * merra2_tiaret['toa_radiation'] * np.sqrt(
    merra2_tiaret['T2MMAX'] - merra2_tiaret['T2MMIN']) * (merra2_tiaret['T2MMEAN'] + 17.8)

<div dir="rtl" style="text-align:right">

### إعادة أخذ عينات بيانات CHIRPS

</div>


<div dir="rtl" style="text-align:right">

أخيرًا نحتاج إلى معالجة بيانات الهطول الشهرية من <bdi dir="ltr">CHIRPS</bdi> وتحويلها إلى هطول يومي لمنطقة تيارت كما فعلنا في الدرس السابق.

</div>


In [ ]:
chirps_tiaret = ds_chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret_daily = chirps_tiaret.resample(time = 'D').nearest().mean(['x', 'y']) / 30
chirps_tiaret_daily

<div dir="rtl" style="text-align:right">

### اختيار جزء من سلسلة زمنية في xarray

الآن نحن جاهزون لمقارنة عامي <bdi dir="ltr">2023</bdi> و<bdi dir="ltr">2024</bdi>.

إذا كان مجلد <bdi dir="ltr">data_raw/MERRA2</bdi> يحتوي على بيانات العامين، فإن <bdi dir="ltr">ds_merra2</bdi> سيحتوي على السلسلة الزمنية لكليهما. رأينا كيف نختار تاريخًا محددًا عبر <bdi dir="ltr">sel(time = 'YYYY-MM-DD')</bdi>، لكن كيف نختار كل بيانات سنة أو شهر أو نطاق زمني؟

كل بُعد في <bdi dir="ltr">xarray</bdi> من نوع <bdi dir="ltr">datetime64[ns]</bdi> لديه مُلحق زمني <bdi dir="ltr">DatetimeAccessor</bdi> عبر الخاصية <bdi dir="ltr">dt</bdi>.

</div>


In [ ]:
merra2_tiaret_pet.time.dt

<div dir="rtl" style="text-align:right">

<bdi dir="ltr"><a href="https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html">The dt property can be used to subset a time series.</a></bdi>

</div>


In [ ]:
merra2_tiaret_pet.time.dt.year

<div dir="rtl" style="text-align:right">

مثلًا، لاختيار بيانات سنة <bdi dir="ltr">2023</bdi> فقط...

</div>


In [ ]:
# Select only those data points along the time access where time.year is in a list of years
merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([2023]))

In [ ]:
ratios = []
precip = []
for year in [2023, 2024]:
    pet_this_year = merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([year]))
    precip_this_year = chirps_tiaret_daily.sel(time = chirps_tiaret_daily.time.dt.year.isin([year]))
    # Select the first 122 days
    pet_this_year = pet_this_year.isel(time = slice(0, 122))
    precip_this_year = precip_this_year.isel(time = slice(0, 122))
    # NOTE: We must use the .values attribute because these are from two different datasets
    ratios.append(100 * precip_this_year.values / pet_this_year.values)
    precip.append(precip_this_year.values)

In [ ]:
pyplot.figure(figsize = (12, 5))
for y, year in enumerate([2023, 2024]):
    pyplot.plot(pet_this_year.time, ratios[y], label = year)
    
pyplot.ylabel('Precipitation-to-PET Ratio (mm day-1)')
pyplot.legend()

<div dir="rtl" style="text-align:right">

**يوحي الرسم أعلاه أن الظروف الهيدرولوجية في بداية <bdi dir="ltr">2024</bdi> ليست مختلفة كثيرًا عن العام السابق.** لكن لماذا أدّت هذه الظروف الجافة موسميًا إلى **جفاف اجتماعي-اقتصادي** في <bdi dir="ltr">2024</bdi> ولم تؤدِّ إلى ذلك في <bdi dir="ltr">2023</bdi>؟ كما رأينا عند رسم نسب الهطول إلى <bdi dir="ltr">PET</bdi> على المدى الطويل باستخدام <bdi dir="ltr">TerraClimate</bdi>، كان هطول موسم الرطوبة أقل من المعتاد لعدة سنوات. تفسير محتمل هو أن الأثر **التراكمي** لعدة سنوات ضعيفة الهطول جعل ظروف <bdi dir="ltr">2024</bdi> أشد، حتى لو لم تكن الفروقات واضحة عند مقارنة سنة بسنة.

يمكننا تصور الانخفاض طويل الأمد في هطول الشتاء مرة أخرى باستخدام <bdi dir="ltr">CHIRPS</bdi>. رغم التحسن النسبي في <bdi dir="ltr">2022</bdi>، يبدو أن هطول الشتاء كان منخفضًا بشكل غير معتاد منذ <bdi dir="ltr">2019</bdi>.

</div>


In [ ]:
chirps_tiaret.sel(time = chirps_tiaret.time.dt.month.isin([1,2,3,4,5])).groupby('time.year').sum().mean(['x', 'y']).plot()
pyplot.ylabel('January - May Precipitation (mm)')

---

<div dir="rtl" style="text-align:right">

## موارد إضافية

- <bdi dir="ltr"><a href="https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html">xarray.core.accessor_dt.DatetimeAccessor (Documentation)</a></bdi>

</div>
